In [ ]:
%load_ext autoreload
%autoreload 1
import sys
sys.path.append('../src')

In [ ]:
import IPython.display as ipd
%aimport ponysynth.horse_noises
from ponysynth.horse_noises import *

sc = SpeechCorpus('/home/celestia/data/audio-tgz/Luna.tgz')

In [ ]:
sc.build_phone_index()

In [ ]:
%aimport ponysynth.horse_noises
%autoreload 1
from ponysynth import horse_noises


for sample in sc.triphones().sample(2):
    audio, rate = sample.audio()
    print(len(audio))
    ipd.display(ipd.Audio(audio, rate=rate))
    
    for ph in sample.phonemes():
        print('tri:', ph)
    
    for diph in sample.diphonemes():
        print('di:', diph)
        
        


In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

from ponysynth.models.triphones import *

In [5]:
import tensorflow as tf
import sonnet as snt
tf.__version__

'2.0.0-rc0'